**[SQL Micro-Course Home Page](https://www.kaggle.com/learn/SQL)**

---


# Introduction

With all that you've learned, your SQL queries are getting pretty long, which can make them hard understand (and debug).

You are about to learn how to use **AS** and **WITH** to tidy up your queries and make them easier to read.

Along the way, we'll use the familiar `pets` table, but now it includes the ages of the animals. 

![](https://i.imgur.com/MXrsiAZ.png)

# AS

You learned in an earlier tutorial how to use **AS** to rename the columns generated by your queries, which is also known as **aliasing**.  This is similar to how Python uses `as` for aliasing when doing imports like `import pandas as pd` or `import seaborn as sns`.  

To use **AS** in SQL, insert it right after the column you select.  Here's an example of a query _without_ an **AS** clause:

![](https://i.imgur.com/VelX9tP.png)

And here's an example of the same query, but _with_ **AS**.

![](https://i.imgur.com/teF84tU.png)

These queries return the same information, but in the second query the column returned by the **COUNT()** function will be called `Number`, rather than the default name of `f0__`.

# WITH ... AS

On its own, **AS** is a convenient way to clean up the data returned by your query. It's even more powerful when combined with **WITH** in what's called a "common table expression".

A **common table expression** (or **CTE**) is a temporary table that you return within your query. CTEs are helpful for splitting your queries into readable chunks, and you can write queries against them. 

For instance, you might want to use the `pets` table to ask questions about older animals in particular.  So you can start by creating a CTE which only contains information about animals more than five years old like this:

![](https://i.imgur.com/0Kz8q4x.png)
            
While this incomplete query above won't return anything, it creates a CTE that we can then refer to (as `Seniors`) while writing the rest of the query.  

We can finish the query by pulling the information that we want from the CTE.  The complete query below first creates the CTE, and then returns all of the IDs from it. 

![](https://i.imgur.com/3xQZM4p.png)
    
You could do this without a CTE, but if this were the first part of a very long query, removing the CTE would make it much harder to follow.

Also, it's important to note that CTEs only exist inside the query where you create them, and you can't reference them in later queries.  So, any query that uses a CTE is always broken into two parts: (1) first, we create the CTE, and then (2) we write a query that uses the CTE.

# Example: How many Bitcoin transactions are made per month?

We're going to use a CTE to find out how many Bitcoin transactions were made each day for the entire timespan of a bitcoin transaction dataset.

We'll investigate the `transactions` table. Here is a view of the first few rows.  (_The corresponding code is hidden, but you can un-hide it by clicking on the "Code" button below._)

In [1]:

from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "crypto_bitcoin" dataset
dataset_ref = client.dataset("crypto_bitcoin", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "transactions" table
table_ref = dataset_ref.table("transactions")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "transactions" table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,hash,size,virtual_size,version,lock_time,block_hash,block_number,block_timestamp,block_timestamp_month,input_count,output_count,input_value,output_value,is_coinbase,fee,inputs,outputs
0,b7356cfb9f77b317c171c5354c3e97dd2a9ec9d3df3763...,134,134,1,0,00000000fe47c494b4ff5393c885888f6e1b134c09ab42...,1758,2009-01-25 10:16:30+00:00,2009-01-01,0,1,None,5000000000,True,0,[],"[{'index': 0, 'script_asm': '04a3c71c9e8d795d6..."
1,af5b4c9417d05229355f0a7ca74fdb6f320b2182f61cfd...,134,134,1,0,0000000073b026aae76ee20013acea08aa45bb75a37947...,685,2009-01-16 09:10:41+00:00,2009-01-01,0,1,None,5000000000,True,0,[],"[{'index': 0, 'script_asm': '048131c6ac22859ff..."
2,0b086abe2602f870a2d93ed496b0558d5fc349c4504eb0...,134,134,1,0,000000009df396001cae68e4bbd0d1073e9056e82794a0...,407,2009-01-14 04:33:00+00:00,2009-01-01,0,1,None,5000000000,True,0,[],"[{'index': 0, 'script_asm': '04d29fcb55e10798c..."
3,634ee14bdfe2bbeee3bfb082f751f1e0603a5735ac880b...,134,134,1,0,0000000089c680170ab1ea7f17a4df3ee03ee6d66a3c84...,725,2009-01-16 18:37:13+00:00,2009-01-01,0,1,None,5000000000,True,0,[],"[{'index': 0, 'script_asm': '04e0987e3cc0ddb24..."
4,3b5237257da3efe8eb36276b37469f219f074bc9b5607a...,134,134,1,0,00000000d3d00462bebfb057d6c3eb931a7690447a31b1...,682,2009-01-16 08:22:35+00:00,2009-01-01,0,1,None,5000000000,True,0,[],"[{'index': 0, 'script_asm': '04df4859b2f06180d..."


Since the `block_timestamp` column contains the date of each transaction in DATETIME format, we'll convert these into DATE format using the **DATE()** command.

We do that using a CTE, and then the next part of the query counts the number of transactions for each date and sorts the table so that earlier dates appear first. 

In [2]:
# Query to select the number of transactions per date, sorted by date
query_with_CTE = """ 
                 WITH time AS 
                 (
                     SELECT DATE(block_timestamp) AS trans_date
                     FROM `bigquery-public-data.crypto_bitcoin.transactions`
                 )
                 SELECT COUNT(1) AS transactions,
                        trans_date
                 FROM time
                 GROUP BY trans_date
                 ORDER BY trans_date
                 """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
query_job = client.query(query_with_CTE, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
transactions_by_date = query_job.to_dataframe()

# Print the first five rows
transactions_by_date.head()

BadRequest: 400 GET https://dp.kaggle.net/bigquery/v2/projects/kaggle-161607/queries/942fb782-685c-43b6-b3d1-9bd9e9436565?maxResults=0&location=US: Query exceeded limit for bytes billed: 1000000000. 3331325952 or higher required.

Since they're returned sorted, we can easily plot the raw results to show us the number of Bitcoin transactions per day over the whole timespan of this dataset.

In [3]:
transactions_by_date.set_index('trans_date').plot()

NameError: name 'transactions_by_date' is not defined

As you can see, common table expressions (CTEs) let you shift a lot of your data cleaning into SQL. That's an especially good thing in the case of BigQuery, because it is vastly faster than doing the work in Pandas.

# Your turn
You now have the tools to stay organized even when writing more complex queries.  Now **[use them here](https://www.kaggle.com/kernels/fork/682113)**.


---
**[SQL Micro-Course Home Page](https://www.kaggle.com/learn/SQL)**

